In [2]:
from deepface import DeepFace
import os
import pandas as pd

2025-10-20 16:02:11.881075: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 16:02:12.453350: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 16:02:14.011817: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [12]:
dfs = DeepFace.find(img_path="../public/albums/mic_2025_amiens/100_2019.JPG", db_path="../public/albums/mic_2025_amiens", enforce_detection=False)
print(dfs)

25-10-20 16:35:10 - Searching ../public/albums/mic_2025_amiens/100_2019.JPG in 282 length datastore
25-10-20 16:35:10 - find function duration 1.6962215900421143 seconds
[                                             identity  \
0       ../public/albums/mic_2025_amiens/100_2019.JPG   
1       ../public/albums/mic_2025_amiens/100_2010.JPG   
2   ../public/albums/mic_2025_amiens/test/100_2010...   
3       ../public/albums/mic_2025_amiens/100_2099.JPG   
4       ../public/albums/mic_2025_amiens/100_2006.JPG   
5   ../public/albums/mic_2025_amiens/test/100_2006...   
6       ../public/albums/mic_2025_amiens/100_2095.JPG   
7       ../public/albums/mic_2025_amiens/100_2125.JPG   
8       ../public/albums/mic_2025_amiens/100_2021.JPG   
9       ../public/albums/mic_2025_amiens/100_2140.JPG   
10      ../public/albums/mic_2025_amiens/100_2038.JPG   
11      ../public/albums/mic_2025_amiens/100_2122.JPG   

                                        hash  target_x  target_y  target_w  \
0   a8b0a

In [ ]:
import cv2
import numpy as np

df = dfs[0]

for i, row in df.iterrows():
    img_path = row["identity"]
    # print("Chemin absolu :", os.path.abspath(img_path))
    img = cv2.imread(img_path)

    if img is None:
        print("⚠️ Impossible de lire :", img_path)
        continue

    # Récupération et validation des coordonnées
    x, y, w, h = int(row["target_x"]), int(row["target_y"]), int(row["target_w"]), int(row["target_h"])

    # Sécurité : s'assure que le rectangle ne dépasse pas l'image
    h_img, w_img = img.shape[:2]
    x = max(0, min(x, w_img - 1))
    y = max(0, min(y, h_img - 1))
    w = max(1, min(w, w_img - x))
    h = max(1, min(h, h_img - y))

    # ✅ Rectangle noir
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)

    # Label de confiance
    label = f"{row['confidence']:.1f}%"
    cv2.putText(img, label, (x, max(30, y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

    cv2.imshow("Visage", cv2.resize(np.array(img, dtype=np.uint8), None, fx=.25, fy=.25))

    key = cv2.waitKey(0)
    if key == 27:  # Échap pour quitter
        break

cv2.destroyAllWindows()